In [1]:
import joblib
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_transformer

import os
import pandas as pd
import numpy as np

# store elements as dictionary keys and their counts as dictionary values
from collections import Counter

# scikit-learn
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline

# Classification metrics
from sklearn.metrics import confusion_matrix, classification_report

# Function for creating model pipelines - sklearn
from sklearn.pipeline import make_pipeline

# Function for creating model pipelines - imblearn
# from imblearn.pipeline import make_pipeline as imbl_pipe

# Over-sampling using SMOTE
from imblearn.over_sampling import SMOTE
# Import classifier
from sklearn.linear_model import LogisticRegression
import datetime as dt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
logistic_regression = joblib.load('./Models/Logistic_regression.sav')

In [3]:
gradient_boosting = joblib.load('./Models/Gradient_Boosting.sav')

In [4]:
random_forest = joblib.load('./Models/Random_Forest.sav')

In [5]:
df_filtered = pd.read_parquet('df_filtered_corr.parquet')
new_data = df_filtered.sample(1000,random_state=100)

In [6]:
new_data.head()

,CLIENT_NO,CIF_OPEN_CHANNEL,EBANK_FIRST_OPEN_CHANNEL,EBANK_CHANNEL_GROUP,GENDER,AGE,MB2_FIRST_CHANNEL,MB2_CHANNEL_GROUP,EBANK_FLAG,LOGIN_MB2_DAY_CNT_1M,...,AR_CNT_3M,PIGGY_CNT_3M,VOICEBOT_CNT_3M,VIB_CARE_INSURANCE_CNT_3M,MORE_USAGE_DAY_CNT_3M,AR_CNT_1M,PIGGY_CNT_1M,VOICEBOT_CNT_1M,VIB_CARE_INSURANCE_CNT_1M,Y
521286,3535329,MYVIB,MYVIB,MYVIB,FEMALE,24.0,MIG,MIGRATE,1,0,...,0,0,0,0,0,0,0,0,0,1
269398,4247650,CREDIT CARD,AUTO_JOB,CREDIT CARD,MALE,39.0,AUTO_JOB,CREDIT CARD,1,4,...,0,0,0,0,1,0,0,0,0,0
879716,3578669,CREDIT CARD,AUTO_JOB,CREDIT CARD,MALE,37.0,MIG,MIGRATE,1,1,...,0,0,0,0,1,0,0,0,0,1
1583736,3915235,SMART SALES,SS,SMART SALES,MALE,22.0,MIG,MIGRATE,1,0,...,0,0,0,0,0,0,0,0,0,1
1344873,4217862,CREDIT CARD,AUTO_JOB,CREDIT CARD,MALE,40.0,AUTO_JOB,CREDIT CARD,1,4,...,0,0,0,0,1,0,0,0,0,0


In [7]:
# Object for target variable
y = new_data.Y

# object for input features
X = new_data.drop(['CLIENT_NO', 'Y'], axis=1)

In [8]:
X

,CIF_OPEN_CHANNEL,EBANK_FIRST_OPEN_CHANNEL,EBANK_CHANNEL_GROUP,GENDER,AGE,MB2_FIRST_CHANNEL,MB2_CHANNEL_GROUP,EBANK_FLAG,LOGIN_MB2_DAY_CNT_1M,LOGIN_MB1_DAY_CNT_1M,...,SECURITY_CNT_3M,AR_CNT_3M,PIGGY_CNT_3M,VOICEBOT_CNT_3M,VIB_CARE_INSURANCE_CNT_3M,MORE_USAGE_DAY_CNT_3M,AR_CNT_1M,PIGGY_CNT_1M,VOICEBOT_CNT_1M,VIB_CARE_INSURANCE_CNT_1M
521286,MYVIB,MYVIB,MYVIB,FEMALE,24.0,MIG,MIGRATE,1,0,0,...,0,0,0,0,0,0,0,0,0,0
269398,CREDIT CARD,AUTO_JOB,CREDIT CARD,MALE,39.0,AUTO_JOB,CREDIT CARD,1,4,0,...,0,0,0,0,0,1,0,0,0,0
879716,CREDIT CARD,AUTO_JOB,CREDIT CARD,MALE,37.0,MIG,MIGRATE,1,1,0,...,1,0,0,0,0,1,0,0,0,0
1583736,SMART SALES,SS,SMART SALES,MALE,22.0,MIG,MIGRATE,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1344873,CREDIT CARD,AUTO_JOB,CREDIT CARD,MALE,40.0,AUTO_JOB,CREDIT CARD,1,4,0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655298,SMART SALES,SS,SMART SALES,FEMALE,30.0,SS,SMART SALES,1,20,0,...,0,0,0,0,0,1,0,0,0,0
1104569,CREDIT CARD,AUTO_JOB,CREDIT CARD,FEMALE,23.0,AUTO_JOB,CREDIT CARD,1,2,0,...,0,0,0,0,0,1,0,0,0,0
1393140,SMART SALES,SS,SMART SALES,MALE,19.0,SS,SMART SALES,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1416193,MYVIB,MYVIB,MYVIB,MALE,19.0,MIG,MIGRATE,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
y

521286     1
269398     0
879716     1
1583736    1
1344873    0
          ..
655298     0
1104569    0
1393140    1
1416193    1
411186     1
Name: Y, Length: 1000, dtype: int64

In [10]:
num_columns = X.select_dtypes(include='number').columns.tolist()
cat_columns = X.select_dtypes(include='object').columns.tolist()
num_features = []

for i in num_columns:
    location = X.columns.get_loc(i)
    num_features.append(location)
print(num_features)

cat_features = []

for i in cat_columns:
    location = X.columns.get_loc(i)
    cat_features.append(location)
print(cat_features)

[4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]
[0, 1, 2, 3, 5, 6]


In [11]:
# Define column transformer
# Need to be numeric not string to specify columns name 
preprocess = make_column_transformer(
    (MinMaxScaler(), num_features),
    (OneHotEncoder(sparse=False), cat_features)
)
preprocess

ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                 [4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                                  18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
                                  29, 30, 31, 32, 33, 34, 35, ...]),
                                ('onehotencoder', OneHotEncoder(sparse=False),
                                 [0, 1, 2, 3, 5, 6])])

In [12]:
# Fit preprocessing pipeline on train data
preprocess.fit(X)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                 [4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                                  18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
                                  29, 30, 31, 32, 33, 34, 35, ...]),
                                ('onehotencoder', OneHotEncoder(sparse=False),
                                 [0, 1, 2, 3, 5, 6])])

In [13]:
X_train_preprocessed = preprocess.transform(X)

In [14]:
lr_predictions = logistic_regression.predict(X_train_preprocessed)
gb_predictions = gradient_boosting.predict(X_train_preprocessed)
rf_predictions = random_forest.predict(X_train_preprocessed)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [15]:
lr_accuracy = classification_report(y, lr_predictions)
gb_accuracy = classification_report(y, gb_predictions)
rf_accuracy = classification_report(y, rf_predictions)

print("Logistic Regression Accuracy:\n", lr_accuracy)
print("Gradient Boosting Accuracy:\n", gb_accuracy)
print("Random Forest Accuracy:\n", rf_accuracy)

Logistic Regression Accuracy:
               precision    recall  f1-score   support

           0       0.95      0.86      0.90       355
           1       0.93      0.97      0.95       645

    accuracy                           0.93      1000
   macro avg       0.94      0.92      0.93      1000
weighted avg       0.93      0.93      0.93      1000

Gradient Boosting Accuracy:
               precision    recall  f1-score   support

           0       0.88      0.95      0.91       355
           1       0.97      0.93      0.95       645

    accuracy                           0.94      1000
   macro avg       0.93      0.94      0.93      1000
weighted avg       0.94      0.94      0.94      1000

Random Forest Accuracy:
               precision    recall  f1-score   support

           0       0.86      0.95      0.90       355
           1       0.97      0.91      0.94       645

    accuracy                           0.93      1000
   macro avg       0.92      0.93      0.92